In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_ocr.csv', index_col=0, encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_1x256-bidirect.csv', encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_1x256.csv', encoding='utf-8')
df2 = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_ocr-test-lowercase.csv', encoding='utf-8')
df = pd.read_csv('/home/jvdzwaan/data/dncvu/word_mapping_gs_256-seq2seq-lowercase.csv', encoding='utf-8')
#df = pd.read_csv('/home/jvdzwaan/data/tmp/wm/BAOku56.csv')
print df.shape
#print df2.shape
df

In [ ]:
def to_list_of_dfs(df):
    dfs = []
    start = 0
    for i in df.index[1:]:
        res = df.loc[i]
        if res['Unnamed: 0'] == 0:
            dfs.append(df.loc[start:i-1])
            start = i
    dfs.append(df.loc[start:])
    return dfs

def merge_ocr(wm_improved_ocr, wm_original_ocr):
    io_dfs = to_list_of_dfs(wm_improved_ocr)
    oo_dfs = to_list_of_dfs(wm_original_ocr)

    res = []
    for d1, d2 in zip(io_dfs, oo_dfs):
        d1 = d1.reset_index()
        del d1['index']
        d2 = d2.reset_index()
        del d2['index']
        r = pd.concat([d1, d2], axis=1)
        r.columns = ['wordindex1', 'gs', 'ocr', 'wordindex2', 'gs2', 'original-ocr']
        res.append(r[['gs', 'ocr', 'original-ocr', 'gs2']])
    df = pd.concat(res)
    return df

df = merge_ocr(df, df2)
print df.shape

In [ ]:
df

In [ ]:
from ochre.ocrerrors import hyphenation_error, hyphenation_error2, 

In [ ]:
df[df['gs'] != df['ocr']].shape

In [ ]:
def pr(row):
    print(repr(row['gs']), repr(row['gs2']))

df[df['gs'] != df['gs2']].apply(pr, axis=1)

In [ ]:
errors = df[df['gs'] != df['ocr']]
errors = errors.fillna(u'@@@')
print errors.shape
#errors

In [ ]:
errors

In [ ]:
def hyphenation_error(row):
    #ocr = 'gesta- tionneerd'
    #gs = 'gestationneerd'
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if ocr.replace(u'- ', u'') == gs:
        return True
    return False

#print errors[errors.apply(hyphenation_error, axis=1)]
errors[errors.apply(hyphenation_error, axis=1)].shape

In [ ]:
e = {}
e['hyphenation'] = errors[errors.apply(hyphenation_error, axis=1)].shape[0]
e

In [ ]:
errors = errors[errors.apply(hyphenation_error, axis=1) == False]
errors.shape

In [ ]:
def hyphenation_error2(row):
    #ocr = 'gesta-tionneerd'
    #gs = 'gestationneerd'
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if ocr.replace(u'-', u'') == gs:
        return True
    return False

s = errors[errors.apply(hyphenation_error2, axis=1)].shape
print s
e['hyphenation2'] = s[0]

In [ ]:
errors = errors[errors.apply(hyphenation_error2, axis=1) == False]
errors.shape

In [ ]:
def uppercase_error(row):
    #ocr = 'Woor'
    #gs = 'WOORD'
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if ocr.lower() == gs.lower():
        return True
    return False
s = errors[errors.apply(uppercase_error, axis=1)].shape
print s
e['case'] = s[0]

In [ ]:
errors = errors[errors.apply(uppercase_error, axis=1)==False]
print errors.shape

In [ ]:
from string import punctuation

def punctuation_error(row):
    #ocr = '.'
    #gs = ','
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if ocr in punctuation and gs in punctuation:
        return True
    return False
s = errors[errors.apply(punctuation_error, axis=1)].shape
print s
e['punctuation'] = s[0]
#print errors

In [ ]:
errors = errors[errors.apply(punctuation_error, axis=1)==False]
print errors.shape

In [ ]:
def missing_punctuation_error(row):
    #ocr = ''
    #gs = ','
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if gs in punctuation and ocr == u'@@@':
        return True
    return False
s = errors[errors.apply(missing_punctuation_error, axis=1)].shape
print s
e['missing_punctuation'] = s[0]

In [ ]:
errors = errors[errors.apply(missing_punctuation_error, axis=1)==False]
print errors.shape

In [ ]:
def missing_word_error(row):
    #ocr = ''
    #gs = ','
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if ocr == u'@@@':
        #print gs
        return True
    return False
s = errors[errors.apply(missing_word_error, axis=1)].shape
print s
e['missing word'] = s[0]

In [ ]:
errors = errors[errors.apply(missing_word_error, axis=1)==False]
print errors.shape

In [ ]:
def number_error(row):
    #ocr = '64-jarige'
    #gs = '-jarig'
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if filter(lambda x: not x.isdigit(), ocr) == filter(lambda x: not x.isdigit(), gs):
        return True
    return False
#number_error('bla')
s = errors[errors.apply(number_error, axis=1)].shape
print s
e['number'] = s[0]

In [ ]:
errors = errors[errors.apply(number_error, axis=1)==False]
print errors.shape

In [ ]:
def whitespace_error(row):
    #ocr = 'Wo or'
    #gs = 'WOORD'
    gs = row['gs']
    ocr = row['ocr']
    
    #print type(ocr)
    #print type(gs)
    
    if u''.join(ocr.split()) == u''.join(gs.split()):
        return True
    return False
s = errors[errors.apply(whitespace_error, axis=1)].shape
print s
e['whitespace'] = s[0]

In [ ]:
errors[errors.apply(whitespace_error, axis=1)]

In [ ]:
errors = errors[errors.apply(whitespace_error, axis=1)==False]
s = errors.shape
#e['other'] = s[0]

In [ ]:
import unicodedata

def accent_error(row):
    #gs = u'décors'
    #ocr = u'decors'
    
    gs = row['gs']
    ocr = row['ocr']
    
    no_accents = ''.join((c for c in unicodedata.normalize('NFD', gs) if unicodedata.category(c) != 'Mn'))
    if no_accents == ocr:
        return True
    return False

s = errors[errors.apply(accent_error, axis=1)].shape
e['accent'] = s[0]

In [ ]:
errors = errors[errors.apply(accent_error, axis=1)==False]

In [ ]:
import codecs

with codecs.open('/home/jvdzwaan/Desktop/BWNT05_3.0.1/BWNT05_v003.txt', 'rb', 'latin1') as f:
    lines = f.readlines()

terms = []
    
for line in lines:
    entry = line.split(';')
    # lexicon service needs lower case input
    term = entry[3].lower()
    term = term.replace('"', '')
    terms.append(term)

print len(terms)
terms = set(terms)

In [ ]:
print terms

In [ ]:
def real_word_error(row, ts):
    #gs = u'hallo'
    #ocr = u'boom'
    #print len(ts)
    
    gs = row['gs']
    ocr = row['ocr']
    
    return ocr in terms
s = errors[errors.apply(real_word_error, args=(terms,), axis=1)].shape
print s
e['real_word'] = s[0]

In [ ]:
errors = errors[errors.apply(real_word_error, axis=1)==False]

In [ ]:
s = errors.shape
e['other'] = s[0]

In [ ]:
df = pd.DataFrame(e.items(), columns=['name','value'])
df = df.set_index('name')
df.columns = ['number']
df
df.plot.pie(y='number', figsize=(10,10))

In [ ]:
df.plot.bar(y='number')

In [ ]:
df

In [ ]:
errors

In [ ]:
gr = errors.groupby('gs')
gr2 = errors.groupby('ocr')
print len(errors.groupby('gs').groups)
print len(errors.groupby('ocr').groups)

In [ ]:
single = []

for g in gr2.groups:
    d = gr2.get_group(g)
    if len(d) == 1:
        single.append((g, d['gs'].values[0]))
    else:
        #print g
        words = d['gs'].unique()
        #for w in words:
        #    print w, align(g, w)
        print g, ':', u'\t'.join(words), len(d['gs'])
        #print
        #pass
print len(single)

In [ ]:
import edlib
import unicodedata
from ochre.char_align import align_characters

def align_output_to_input(input_str, output_str, empty_char=u'@'):
    #print type(input_str)
    #print type(output_str)
    #print
    # remove accented and other special characters, because edlib doesn't like them (this might be a python 2 problem)
    #print output_str.encode('ASCII', 'replace')
    #print input_str.encode('ASCII', 'replace')
    t_output_str = output_str.encode('ASCII', 'replace')
    t_input_str = input_str.encode('ASCII', 'replace')
    #print t_output_str, len(t_output_str)
    #print t_input_str, len(t_input_str)
    #print
    try:
        r = edlib.align(t_input_str, t_output_str, task='path')
    except:
        print input_str
        print output_str
    r1, r2 = align_characters(input_str, output_str, r.get('cigar'), empty_char=empty_char, sanity_check=False)
    #print r1
    #print r2
    #print 
    #print u''.join(r1)
    #print u''.join(r2)
    #print 
    #print len(r2)
    #print len(r1)
    while len(r2) < len(input_str):
        r2.append(empty_char)
    return u''.join(r2)

def align(input_str, output_str):
    t_output_str = output_str.encode('ASCII', 'replace')
    t_input_str = input_str.encode('ASCII', 'replace')
    #print t_output_str, len(t_output_str)
    #print t_input_str, len(t_input_str)
    #print
    try:
        r = edlib.align(t_input_str, t_output_str, task='path')
    except:
        print input_str
        print output_str
    return r

In [ ]:
single = []

for g in gr.groups:
    d = gr.get_group(g)
    if len(d) == 1:
        single.append((g, d['ocr'].values[0], d['original-ocr'].values[0]))
    else:
        #print g
        words = d['ocr'].unique()
        #for w in words:
        #    print w, align(g, w)
        print g, ':', u'\t'.join(d['ocr'].unique())
        #print
        #pass
print len(single)

In [ ]:
e['single'] = len(single)

In [ ]:
e['other'] = e['other'] - e['single']

In [ ]:
df = pd.DataFrame(e.items(), columns=['name','number'])
df = df.set_index('name')
df
df.plot.pie(y='number', figsize=(10,10))

In [ ]:
for (g, o, oo) in single:
    print g, o, oo

In [ ]:
errors[errors['ocr']==errors['original-ocr']]

In [ ]:
# hoeveel verbeterd? ocr != original-ocr en ocr == gs en gs != orginal-ocr
errors
#d = errors[errors['ocr']!=errors['original-ocr']]
#d[d['ocr'] == d['gs']]
#d

In [ ]:
# hoeveel verfouterd? ocr != gs en original-ocr == gs